In [7]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from pprint import  pprint
from fake_useragent import  UserAgent
from src.data_fetcher.client import  PriceHistory
from sklearn.preprocessing import  StandardScaler

# Set display options for  pandas
pd.set_option('display.max_colwidth', 1)
pd.set_option('expand_frame_repr', False)

In [9]:
# Define the symbols
symbols = ['AAPL', 'MSFT', 'TSLA', 'AMZN']

# Grab number of stocks in portfolio
number_of_stocks = len(symbols)

# If no data is available, grab it from the NASDAQ.
if not pathlib.Path('src/data/stock_data.csv').exists():
    # Initialize PriceHistory Client
    price_history_client = PriceHistory(symbols=symbols, user_agent=UserAgent().chrome)
    # Grab and dump data in to a csv file.
    price_history_client.price_data_frame.to_csv('src/data/stock_data.csv', index=False)
    pprint(price_history_client.price_data_frame)
    
    # store dataframe in variable
    price_df: pd.DataFrame = price_history_client.price_data_frame
    
else:
    # load the existing CSV file
    price_df: pd.DataFrame = pd.read_csv('src/data/stock_data.csv')
    
display(price_df.head())

,date,close,volume,open,high,low,symbol
0,2025-04-03,178.41,95553620,182.995,184.13,176.920,AMZN
1,2025-04-02,196.01,53679200,187.660,198.34,187.660,AMZN
2,2025-04-01,192.17,41267320,187.860,193.93,187.200,AMZN
3,2025-03-31,190.26,63547560,188.190,191.33,184.400,AMZN
4,2025-03-28,192.72,52548230,198.420,199.26,191.881,AMZN


In [12]:
# Grab important columns
price_df = price_df[['date', 'symbol', 'close']]

# Pivot dataframe to make symbols headers
price_df = price_df.pivot(index='date', columns='symbol', values='close')

display(price_df.head())

symbol,AAPL,AMZN,MSFT,TSLA
date,,,,
2024-10-07,221.69,180.80,409.54,240.83
2024-10-08,225.77,182.72,414.71,244.50
2024-10-09,229.54,185.17,417.46,241.05
2024-10-10,229.04,186.65,415.84,238.77
2024-10-11,227.55,188.82,416.32,217.80


In [24]:
# Calculate log returns
log_return = np.log(1 + price_df.pct_change())

# Generate random weights and rebalance so that they sum up to 1
random_weights = np.array(np.random.random(number_of_stocks))
rebalance_weights = random_weights / np.sum(random_weights)

# Calculate expected returns annualized
exp_ret = np.sum((log_return.mean() * rebalance_weights)) * 252

# Calculate expected volatility annualized
exp_vol  = np.sqrt(
    np.dot(rebalance_weights.T, np.dot(log_return.cov() * 252, rebalance_weights))
)
# Calculate Sharpe Ratio.
sharpe_ratio = (exp_ret - .02)/ exp_vol

In [25]:
weights_df = pd.DataFrame(data={
    'random_weights':random_weights,
    'rebalance_weights':rebalance_weights
})
print('')
print('='*80)
print('PORTFOLIO WEIGHTS:')
print(weights_df)
print('-'*80)

metrics_df = pd.DataFrame(data={
    'Expected Portfolio Return': exp_ret,
    'Expected Portfolio Volatility': exp_vol,
    'Portfolio Sharpe Ratio': sharpe_ratio
}, index=[0])
print('')
print('='*80)
print('PORTFOLIO METRICS:')
print(metrics_df)
print('-'*80)



PORTFOLIO WEIGHTS:
   random_weights  rebalance_weights
0  0.592389        0.232884         
1  0.757194        0.297673         
2  0.459651        0.180701         
3  0.734475        0.288742         
--------------------------------------------------------------------------------

PORTFOLIO METRICS:
   Expected Portfolio Return  Expected Portfolio Volatility  Portfolio Sharpe Ratio
0 -0.022722                   0.336126                      -0.1271                
--------------------------------------------------------------------------------
